In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from nflows.flows import MaskedAutoregressiveFlow

from counterfactuals.datasets import (
    AdultDataset,
    GermanCreditDataset,
    CompasDataset,
    HelocDataset,
    LawDataset,
    MoonsDataset,
)

from counterfactuals.optimizers.base import BaseCounterfactualModel
from counterfactuals.optimizers.approach_three import ApproachThree

from counterfactuals.metrics.metrics import (
    perc_valid_cf,
    perc_valid_actionable_cf,
    continuous_distance,
    categorical_distance,
    distance_l2_jaccard,
    distance_mad_hamming,
    plausibility,
    evaluate_cf,
)

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create dataset

In [4]:
dataset = CompasDataset(file_path="../data/compas_two_years.csv")

In [5]:
# disc_model = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=100)
disc_model = LogisticRegression()
disc_model.fit(dataset.X_train, dataset.y_train.reshape(-1))
print(classification_report(dataset.y_test, disc_model.predict(dataset.X_test)))

              precision    recall  f1-score   support

         0.0       0.79      0.85      0.82       281
         1.0       0.84      0.78      0.81       281

    accuracy                           0.82       562
   macro avg       0.82      0.82      0.82       562
weighted avg       0.82      0.82      0.82       562



# Relabeling

In [6]:
y_pred_train = disc_model.predict(dataset.X_train)
y_pred_test = disc_model.predict(dataset.X_test)
dataset.y_train = y_pred_train
dataset.y_test = y_pred_test

# noise_lvl - zaszumianie numerycznych cech treningowego datasetu
train_dataloader = dataset.train_dataloader(batch_size=128, shuffle=True, noise_lvl=0)
test_dataloader = dataset.test_dataloader(batch_size=128, shuffle=False)

# Create flow model

In [7]:
flow = MaskedAutoregressiveFlow(features=dataset.X_train.shape[1], hidden_features=4, num_blocks_per_layer=2, num_layers=1, context_features=1)

# Define custom search step within class

In [8]:
class CustomApproach(BaseCounterfactualModel):
    def search_step(self, x_param, x_origin, context_origin, context_target, **search_step_kwargs):
        alpha = search_step_kwargs.get("alpha", None)
        beta = search_step_kwargs.get("beta", None)
        if alpha is None:
            raise ValueError("Parameter 'alpha' should be in kwargs")
        if beta is None:
            raise ValueError("Parameter 'beta' should be in kwargs")

        dist = torch.linalg.norm(x_origin-x_param, axis=1)

        p_x_param_c_orig = self.model.log_prob(x_param, context=context_origin)
        p_x_param_c_target = self.model.log_prob(x_param, context=context_target)
        p_x_orig_c_orig = self.model.log_prob(x_origin, context=context_origin.flatten()[0].repeat((x_origin.shape[0], 1)))

        p_x_param_c_orig_with_beta = p_x_param_c_orig + beta
        max_inner = torch.nn.functional.relu(p_x_orig_c_orig-p_x_param_c_target)
        max_outer = torch.nn.functional.relu(p_x_param_c_orig_with_beta - p_x_param_c_target)
        loss = dist + alpha * (max_outer + max_inner)
        return loss, dist, max_inner, max_outer

# Create cf class, train and test flow model

In [9]:
cf = CustomApproach(model=flow, checkpoint_path="model.pt", neptune_run=None)

In [10]:
cf.train_model(
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    epochs=10000,
    lr=0.001,
    patience=20,
    eps=1e-3, # eps for patience
)

/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
Epoch 9999, Train: -1.5071, test: -3.2061: 100%|██████████| 10000/10000 [04:47<00:00, 34.77it/s]


In [11]:
cf.test_model(test_loader=test_dataloader)

{'0.0': {'precision': 0.8904109589041096,
  'recall': 0.8609271523178808,
  'f1-score': 0.8754208754208754,
  'support': 302.0},
 '1.0': {'precision': 0.8444444444444444,
  'recall': 0.8769230769230769,
  'f1-score': 0.860377358490566,
  'support': 260.0},
 'accuracy': 0.8683274021352313,
 'macro avg': {'precision': 0.8674277016742771,
  'recall': 0.8689251146204788,
  'f1-score': 0.8678991169557206,
  'support': 562.0},
 'weighted avg': {'precision': 0.8691453116451897,
  'recall': 0.8683274021352313,
  'f1-score': 0.8684612412538284,
  'support': 562.0}}

# Search counterfactuals

In [12]:
search_step_kwargs = {
    "alpha": 20,
    "beta": 0.1,
}
test_dataloader = dataset.test_dataloader(batch_size=16, shuffle=False)
Xs_cf, Xs_orig, ys_orig = cf.search_batch(
    dataloader=test_dataloader,
    epochs=500,
    lr=0.001,
    **search_step_kwargs
)

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:19<00:00,  1.81it/s]


# Evaluate

In [13]:
evaluate_cf(
    disc_model=disc_model,
    X=Xs_orig,
    X_cf=Xs_cf,
    model_returned=np.ones(Xs_cf.shape[0]).astype(bool),
    continuous_features=dataset.numerical_features,
    categorical_features=dataset.categorical_features,
    X_train=dataset.X_train,
    y_train=dataset.y_train,
    X_test=dataset.X_test,
    y_test=dataset.y_test,
)

{'model_returned_smth': 1.0,
 'valid_cf_disc': 0.5195729537366548,
 'dissimilarity_proximity_categorical_hamming': 0.9820205479452054,
 'dissimilarity_proximity_categorical_jaccard': 0.9820205479452054,
 'dissimilarity_proximity_continuous_manhatan': 1.01149155518431,
 'dissimilarity_proximity_continuous_euclidean': 0.47161848841062187,
 'dissimilarity_proximity_continuous_mad': 44.128832385833576,
 'distance_l2_jaccard': 0.7438329201623998,
 'distance_mad_hamming': 21.117199405626444,
 'plausibility': 21.014522688081886,
 'kde_log_density': -0.5961783704183525,
 'sparsity': 0.9904109589041096}